In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os

In [2]:
ratings = pd.read_csv('ratings.csv', usecols=['userId', 'movieId', 'rating'])
movies = pd.read_csv('movies.csv')

# Gradient descent without regularization

In [23]:
def gradientDescent(R, U, V, alpha, lambdaReg, muReg, nFeatures):
    for i in range(R.shape[0]):
        for j in range(R.shape[1]):
            if R[i, j] > 0:
                r_hat_ij = np.dot(U[i, :], V.T[:, j])
                eij = R[i, j] - r_hat_ij
                for q in range(nFeatures):
                    U[i, q] = U[i, q] + alpha * eij * V[j, q]
                    V[j, q] = V[j, q] + alpha * eij * U[i, q]
    return U, V

def computeTotalError(R, U, V):
    error = 0
    for i in range(R.shape[0]):
        for j in range(R.shape[1]):
            if R[i, j] > 0:
                error += R[i, j] - np.dot(U[i, :], V.T[:, j])
    return error


def computeApproximatedMatrix(U, V):
    return np.dot(U, V.T)

In [39]:
R = np.array([
    [5, 3, 0, 1],
    [4, 0, 0, 1],
    [1, 1, 0, 5],
    [1, 0, 0, 4],
    [0, 1, 5, 4],
])

nFeatures = 2
nUsers, nItems = R.shape
alpha = 0.01
lambdaReg = 0
muReg = 0
nIterations = 10000

In [40]:
U = np.random.normal(size=(nUsers, nFeatures))
V = np.random.normal(size=(nItems, nFeatures))
# U = np.random.randint(0, 5, size=(nUsers, nFeatures))
# V = np.random.randint(0, 5, size=(nItems, nFeatures))

for iIter in range(nIterations):
    U, V = gradientDescent(R, U, V, alpha, lambdaReg, muReg, nFeatures)
    error = computeTotalError(R, U, V)
    if (iIter + 1) % 10 == 0:
        print("Iteration", iIter, "- error:", error)
    if error < 0.0001:
        break

Iteration 9 - error: 22.923087350520046
Iteration 19 - error: 8.530770523308263
Iteration 29 - error: 2.0406077564068044
Iteration 39 - error: 0.4536291391178787
Iteration 49 - error: 0.14157982379731648
Iteration 59 - error: 0.10522317461876496
Iteration 69 - error: 0.12176059709741383
Iteration 79 - error: 0.14000149262711015
Iteration 89 - error: 0.15040084428076672
Iteration 99 - error: 0.15354181485197538
Iteration 109 - error: 0.15170011349711932
Iteration 119 - error: 0.14681255757452472
Iteration 129 - error: 0.14020693177042842
Iteration 139 - error: 0.13272915636505533
Iteration 149 - error: 0.1249029053816878
Iteration 159 - error: 0.11704792976568723
Iteration 169 - error: 0.10935692488074955
Iteration 179 - error: 0.10194335504995367
Iteration 189 - error: 0.09487092815387133
Iteration 199 - error: 0.088171862124795
Iteration 209 - error: 0.08185834456726038
Iteration 219 - error: 0.075929827736812
Iteration 229 - error: 0.07037774035392608
Iteration 239 - error: 0.0651885

In [41]:
computeApproximatedMatrix(U,V)

array([[5.00262258, 2.98429805, 9.62947779, 0.99934022],
       [3.99284225, 2.39426664, 7.80047921, 0.99847385],
       [1.03412813, 0.91173079, 4.72913187, 5.00149023],
       [0.98277085, 0.81995404, 4.06236395, 3.9968804 ],
       [1.48335675, 1.11307643, 4.97484211, 4.00742146]])

In [33]:
R

array([[5, 3, 0, 1],
       [4, 0, 0, 1],
       [1, 1, 0, 5],
       [1, 0, 0, 4],
       [0, 1, 5, 4]])